In [43]:
import duckdb
import pathlib

connection = duckdb.connect("../data/duckdb/database.db", read_only=False)

In [44]:
BASE_PATH = pathlib.Path.cwd().parent / "data" / "raw" / "parquet_files" / "train"

BASE_TRAIN_PATH = BASE_PATH / "train_base.parquet"

APPLPREV0_PATH = BASE_PATH / "train_applprev_1_0.parquet"
APPLPREV1_PATH = BASE_PATH / "train_applprev_1_1.parquet"

DEBITCARD_PATH = BASE_PATH / "train_debitcard_1.parquet"
DEPOSIT_PATH = BASE_PATH / "train_deposit_1.parquet"
OTHER_PATH = BASE_PATH / "train_other_1.parquet"
PERSON_PATH = BASE_PATH / "train_person_1.parquet"

## Load data

In [45]:
base_train: duckdb.DuckDBPyRelation = connection.sql(f"""
SELECT 
    case_id,
    date_decision::DATE as date_decision,
FROM
    '{str(BASE_TRAIN_PATH)}'
""").set_alias("base")

base_train.columns

['case_id', 'date_decision']

In [46]:
deposit_data: duckdb.DuckDBPyRelation = connection.sql(f"""
SELECT
    case_id,
    num_group1::TINYINT as num_group1,
    amount_416A::FLOAT as amount_416A,
    contractenddate_991D::DATE as contractenddate_991D,
    openingdate_313D::DATE as openingdate_313D
FROM
    '{str(DEPOSIT_PATH)}'
""").set_alias("deposit")
deposit_data.columns

['case_id',
 'num_group1',
 'amount_416A',
 'contractenddate_991D',
 'openingdate_313D']

In [47]:
other_data: duckdb.DuckDBPyRelation = connection.sql(f"""
SELECT
    case_id,
    amtdebitoutgoing_4809440A::FLOAT AS amtdebitoutgoing_4809440A,
    amtdepositbalance_4809441A::FLOAT AS amtdepositbalance_4809441A,
    amtdepositoutgoing_4809442A::FLOAT AS amtdepositoutgoing_4809442A,
    amtdebitincoming_4809443A::FLOAT AS amtdebitincoming_4809443A,
    amtdepositincoming_4809444A::FLOAT AS amtdepositincoming_4809444A
FROM
    '{str(OTHER_PATH)}'
""").set_alias("other")

other_data.columns

['case_id',
 'amtdebitoutgoing_4809440A',
 'amtdepositbalance_4809441A',
 'amtdepositoutgoing_4809442A',
 'amtdebitincoming_4809443A',
 'amtdepositincoming_4809444A']

In [48]:
persons: duckdb.DuckDBPyRelation = connection.sql(f"""
SELECT 
    case_id,
    num_group1::TINYINT AS num_group1,
    birth_259D::DATE AS birth_259D,
    incometype_1044T,
    role_1084L,
    empl_employedfrom_271D::DATE AS empl_employedfrom_271D,
    empl_industry_691L,
    mainoccupationinc_384A::FLOAT AS mainoccupationinc_384A
FROM
    '{str(PERSON_PATH)}'
""").set_alias("person")

persons.columns

['case_id',
 'num_group1',
 'birth_259D',
 'incometype_1044T',
 'role_1084L',
 'empl_employedfrom_271D',
 'empl_industry_691L',
 'mainoccupationinc_384A']

In [49]:
applprev: duckdb.DuckDBPyRelation = connection.sql(f"""
SELECT * FROM '{str(APPLPREV0_PATH)}'
UNION
SELECT * FROM '{str(APPLPREV1_PATH)}'
""").set_alias("applprev")

applprev.columns

['case_id',
 'actualdpd_943P',
 'annuity_853A',
 'approvaldate_319D',
 'byoccupationinc_3656910L',
 'cancelreason_3545846M',
 'childnum_21L',
 'creationdate_885D',
 'credacc_actualbalance_314A',
 'credacc_credlmt_575A',
 'credacc_maxhisbal_375A',
 'credacc_minhisbal_90A',
 'credacc_status_367L',
 'credacc_transactions_402L',
 'credamount_590A',
 'credtype_587L',
 'currdebt_94A',
 'dateactivated_425D',
 'district_544M',
 'downpmt_134A',
 'dtlastpmt_581D',
 'dtlastpmtallstes_3545839D',
 'education_1138M',
 'employedfrom_700D',
 'familystate_726L',
 'firstnonzeroinstldate_307D',
 'inittransactioncode_279L',
 'isbidproduct_390L',
 'isdebitcard_527L',
 'mainoccupationinc_437A',
 'maxdpdtolerance_577P',
 'num_group1',
 'outstandingdebt_522A',
 'pmtnum_8L',
 'postype_4733339M',
 'profession_152M',
 'rejectreason_755M',
 'rejectreasonclient_4145042M',
 'revolvingaccount_394A',
 'status_219L',
 'tenor_203L']

In [50]:
data: duckdb.DuckDBPyRelation = base_train.join(
    other_data, how="left", condition="case_id"
).join(
    applprev, how="left", condition="case_id"
).join(
    deposit_data, how="left", condition="case_id, num_group1"
).join(
    persons, how="left", condition="case_id, num_group1"
).set_alias("data")

data.

┌─────────┬───────────────┬──────────────────────┬───┬────────────────────┬──────────────────────┐
│ case_id │ date_decision │ amtdebitoutgoing_4…  │ … │ empl_industry_691L │ mainoccupationinc_…  │
│  int64  │     date      │        float         │   │      varchar       │        float         │
├─────────┼───────────────┼──────────────────────┼───┼────────────────────┼──────────────────────┤
│   40872 │ 2019-12-18    │                 NULL │ … │ OTHER              │              30000.0 │
│   40881 │ 2019-12-17    │                 NULL │ … │ NULL               │                 NULL │
│   40945 │ 2019-12-19    │                 NULL │ … │ NULL               │                 NULL │
│   40964 │ 2019-12-17    │                 NULL │ … │ OTHER              │              90000.0 │
│   40983 │ 2019-12-18    │                 NULL │ … │ NULL               │                 NULL │
│   40985 │ 2019-12-18    │                 NULL │ … │ NULL               │              11200.0 │
│   41047 

In [54]:
database_length: int = data.shape[0]
for col in data.columns:
    count = data.count(col).fetchone()[0]
    print(col, count, round(count / database_length, 3))
    if count / database_length < 1 - 0.85:
        connection.execute(f"""UPDATE data SET {col} = 0 WHERE {col} IS NULL""")
        connection.execute(f"""UPDATE data SET {col} = 1 WHERE {col} IS NOT NULL""")
        


case_id 6831116 1.0
date_decision 6831116 1.0
amtdebitoutgoing_4809440A 510148 0.075


BinderException: Binder Error: Can only update base table!